# Face Scenarios

In [47]:
import asyncio
import io
import glob
import os
import sys
import time
import uuid
import requests
from urllib.parse import urlparse
from io import BytesIO
from PIL import Image, ImageDraw
from azure.cognitiveservices.vision.face import FaceClient
from msrest.authentication import CognitiveServicesCredentials
from azure.cognitiveservices.vision.face.models import TrainingStatusType, Person, SnapshotObjectType, OperationStatusType
from dotenv import load_dotenv

In [49]:
load_dotenv()

# Set the FACE_SUBSCRIPTION_KEY environment variable with your key as the value.
# This key will serve all examples in this document.
KEY = os.getenv('FACE_KEY')

# Set the FACE_ENDPOINT environment variable with the endpoint from your Face service in Azure.
# This endpoint will be used in all examples in this quickstart.
ENDPOINT = os.getenv('FACE_ENDPOINT')

fbd682cf5ed044029d1826418468fc71


## Emotion Detection

In [19]:
# Create an authenticated FaceClient.
face_client = FaceClient(ENDPOINT, CognitiveServicesCredentials(KEY))

# Detect a face in an image that contains a single face
single_face_image_url = 'https://www.biography.com/.image/t_share/MTQ1MzAyNzYzOTgxNTE0NTEz/john-f-kennedy---mini-biography.jpg'
single_image_name = os.path.basename(single_face_image_url)
attributes = ['age', 'emotion']
detected_faces = face_client.face.detect_with_url(url=single_face_image_url,return_face_attributes=attributes)
if not detected_faces:
    raise Exception('No face detected from image {}'.format(single_image_name))

# Display the detected face ID in the first single-face image.
# Face IDs are used for comparison to faces (their IDs) detected in other images.
print('Detected face ID from', single_image_name, ':')
# Save this ID for use in Find Similar
first_image_face_ID = detected_faces[0].face_id

Detected face ID from john-f-kennedy---mini-biography.jpg :
{'additional_properties': {}, 'age': 52.0, 'gender': None, 'smile': None, 'facial_hair': None, 'glasses': None, 'head_pose': None, 'emotion': <azure.cognitiveservices.vision.face.models._models_py3.Emotion object at 0x7fa0cab133d0>, 'hair': None, 'makeup': None, 'occlusion': None, 'accessories': None, 'blur': None, 'exposure': None, 'noise': None}


In [21]:
for face in detected_faces:
    print(face.face_attributes.emotion)

{'additional_properties': {}, 'anger': 0.0, 'contempt': 0.002, 'disgust': 0.0, 'fear': 0.0, 'happiness': 0.0, 'neutral': 0.997, 'sadness': 0.001, 'surprise': 0.0}


## Finding Match Faces In A Group of Photos

- Get a celebrity face as query face
- Find the matches in a set of group photos

### Detect Query Face

In [32]:
query_face_url = "https://img.srgcdn.com/e//a3VzaG5nZmY4WnBqN21Jb2pUMEQucG5n.jpg"
single_image_name = os.path.basename(query_face_url)
detected_faces = face_client.face.detect_with_url(url=query_face_url)

query_face = detected_faces[0].face_id # query face 

### Find match in a group of photos

In [35]:
multi_face_image_urls = ["https://images.ladbible.com/resize?type=jpeg&url=http://20.theladbiblegroup.com/s3/content/8d062e8358c019b321ef427c313f27a5.png&quality=70&width=720&aspectratio=16:9&extend=white", 
"https://pmchollywoodlife.files.wordpress.com/2015/06/emma-watson-joins-taylor-swifts-crew-ftr-1.jpg", 
"https://images.vogue.it/gallery/40353/Big/d62bc4a8-63e4-44f7-84cd-045ddb1b5605.JPG", 
"https://akm-img-a-in.tosshub.com/indiatoday/images/story/201912/harry_potter-770x433.jpeg?lW8VCYre9okxIKqugT8fbEdwm9e2HXO5"]

lst_fce = {}

for url in multi_face_image_urls: 
    lst = face_client.face.detect_with_url(url)
    lst_fce[url] = lst



In [40]:
def getRectangle(faceDictionary):
    rect = faceDictionary.face_rectangle
    left = rect.left
    top = rect.top
    right = left + rect.width
    bottom = top + rect.height
    
    return ((left, top), (right, bottom))

In [41]:
for url, lst in lst_fce.items(): 
    second_image_face_IDs = list(map(lambda x: x.face_id, lst))
    # Next, find similar face IDs like the one detected in the first image.
    similar_faces = face_client.face.find_similar(face_id=query_face, face_ids=second_image_face_IDs)
    print(f"similar faces for image: {url}")
    for fce in similar_faces: 
        id = fce.face_id
        # The similar face IDs of the single face image and the group image do not need to match, 
        # they are only used for identification purposes in each image.
        # The similar faces are matched using the Cognitive Services algorithm in find_similar().
        face_info = next(x for x in lst if x.face_id == id)
        if face_info:
            print('  Face ID: ', id)
            print('  Face rectangle:')
            print('  Left: ', str(face_info.face_rectangle.left))
            print('  Top: ', str(face_info.face_rectangle.top))
            print('  Width: ', str(face_info.face_rectangle.width))
            print('  Height: ', str(face_info.face_rectangle.height))

            # Download the image from the url
            response = requests.get(url)
            img = Image.open(BytesIO(response.content))
            draw = ImageDraw.Draw(img)
            draw.rectangle(getRectangle(face_info), outline='red')
            
            img.show()
            

similar faces for image: https://images.ladbible.com/resize?type=jpeg&url=http://20.theladbiblegroup.com/s3/content/8d062e8358c019b321ef427c313f27a5.png&quality=70&width=720&aspectratio=16:9&extend=white
  Face ID:  f9218728-1787-4000-91ad-41dc7348ff91
  Face rectangle:
  Left:  324
  Top:  62
  Width:  36
  Height:  36
similar faces for image: https://pmchollywoodlife.files.wordpress.com/2015/06/emma-watson-joins-taylor-swifts-crew-ftr-1.jpg
  Face ID:  e55e8c9a-a307-40e3-a9ed-6aa5e9a213c4
  Face rectangle:
  Left:  266
  Top:  218
  Width:  61
  Height:  61
similar faces for image: https://images.vogue.it/gallery/40353/Big/d62bc4a8-63e4-44f7-84cd-045ddb1b5605.JPG
  Face ID:  f7718808-ab19-46fa-b1de-1ed0a18bcf8c
  Face rectangle:
  Left:  172
  Top:  182
  Width:  67
  Height:  67
similar faces for image: https://akm-img-a-in.tosshub.com/indiatoday/images/story/201912/harry_potter-770x433.jpeg?lW8VCYre9okxIKqugT8fbEdwm9e2HXO5
  Face ID:  fe172541-a6ff-4ec0-be64-a1bff74839cf
  Face rec